# Homework 4

In this assignment you will construct a `DataFrame` with information about the current components of the S&P 500 index.

## Getting data

Use the `pd.read_html` function to download the table of company data on this web site:

https://en.wikipedia.org/wiki/List_of_S%26P_500_companies

Note that the function returns a **list** of DataFrames, so you have to identify the element of that list that contains the DataFrame you want. Store this DataFrame in a variable called `comps`.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
html = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies', header = 0)
comps = html[0]

Display the first 5 rows of `comps`.

In [ ]:
comps.head(5)

## Cleaning

Make the following changes to `comps`:
 - Drop columns *SEC filings*, *GICS Sub Industry*, and *Founded*
 - Rename the remaining columns **name**, **ticker**, **sector**, **address**, **date**, and **cik**
 - Set the index to be **ticker**
 

In [ ]:
comps = comps.drop(columns = ['SEC filings','GICS Sub Industry','Founded'])
comps = comps.rename(index = str, columns = {'Security':'name','Symbol':'ticker','GICS Sector':'sector',
                                     'Headquarters Location':'address','Date first added':'date','CIK':'cik'})
comps = comps.set_index('ticker')

Use `dtypes` to check how each column is stored.

In [ ]:
comps.dtypes

Notice that `date` is stored as text (which pandas lists as *object*). Use the `pd.to_datetime` function to convert the date into a pandas `datetime` object.

In [ ]:
comps['date'] = pd.to_datetime(comps['date'])

## Analysis

It is always a good idea to start by *looking* at your data to get a sense of what it contains and what sort of errors there may be.

Start by asking: How many records are in this dataframe? Use *one* simple function in one line to find out. 

In [ ]:
print('There are {} records in this dataframe.'.format(len(comps)))

Does the answer surprise you? Try to explain it. 

Hint: Are there any CIKs that appear more than once? Why?

In [ ]:
test = comps.cik.value_counts()
test = test.to_frame()
test = test.reset_index()
test = list(test[test['cik'] == 2].loc[:,'index'])
comps[comps['cik'].isin(test)]

# There are CIKs which appear more than once because the company has separated
# its common stock into multiple classes. Please see the output below.

Notice that the date for **MMM** is missing. How many rows are missing the date?

In [ ]:
comps.loc['MMM']

In [ ]:
print('There are {} rows with a missing date.'.format(len(comps[comps['date'].isnull()])))

Add a column called `age` that has the number of years since the company was first added to the index.

Hint: Since `date` is now stored as a `datetime` object, we can access the year using its `.dt.year` property. Just subtract the initial year from 2019.

In [ ]:
comps['age'] = 2019 - comps.date.dt.year

Use the code below to plot a histogram of the `age` column.

In [ ]:
%matplotlib notebook
comps['age'].plot('hist', bins=25)

How long has the average firm been in the index? What about the median? How long has the oldest firm been in the index? All of these questions can be answered with the `.describe()` method.

Note also how many firms you have results for, and comment on what this means.

In [ ]:
print('Descriptive Statistics for Age:\n')
print(comps.age.describe())

print('\nSkewness = {}'.format(comps.age.skew()))

In [ ]:
# Note: Statistics are based on 402 observations (103 missing data points).

# The output above indicates that 50% of the firms in the S&P 500 have been in the index for
# less than or equal to 11 years.  Furthermore, this index exhibits some variability with an
# IQR of 21 years. Lastly, the distribution exhibit a skewness of 0.9527 --> right skewed



What is the earliest `date` in the data?

In [ ]:
d = str(comps.date.min())[:10]
print('The earliest date in the data is {}'.format(d))

Which firms have been in the index since that date?

In [ ]:
print('The following firms have been in the index since {}: {}'.format(d, 
      ', '.join(list(comps[comps['date'] == comps.date.min()].loc[:,'name']))))

How many companies are there in each sector?

In [ ]:
comps.sector.value_counts()

## Working with text data

Use the `.str.extract()` function to create a new column, *state*, with the state name from the *address* column.

Hint: You will need to pass a regular expression that identifies the state. Before coming up with a regular expression look at the examples below:

In [ ]:
comps.loc[['AIG', 'BHF', 'BSX', 'BXP']]

In [ ]:
import re

df_state = comps.address.str.extract(r'\.*?, ([A-Za-z \.]+)', expand = False)
comps['state'] = df_state

How many companies are there headquartered in each state? Do you any problems with your result?

In [ ]:
comps.state.value_counts()

In [ ]:
# For some of the international companies, its address may be structured such that 
# the country is listed where the state normally is (for US based companies).
# Also, the United Kingdom is expressed as both 'United Kingdom' and 'UK'
# Furthermore, Kent and Surrey are counties in the UK. 